# Maxwell–Boltzmann Distribution
## PHAS0024 $\;\;\;\;\;\;\;\;\;$ UCL $\;\;\;\;\;\;\;\;\;$ 2020
### Authors: Dr. Bart Hoogenboom, Lorenzo Braccini

<font color=red>Note: the red texts are meant as side notes during the implementation.

I also included some Markdown cells as explanations of the physics behind the simulations. This also helped me to keep track of the work.
</font> 

## Introduction:

Let us consider a gas (mad of N particles) at a certain temperature T. Assuming that the energy of the particles in the gas is predominantly kinetic. Can we predict the velocity of these particles?

This is an example of **Canonical Distribution**:

$$P(E) = \frac{1}{Z} \cdot \Omega(E) \cdot e^{- \beta E}$$where:
- $P(E)$ is the probabilty of the system to be in the state with energy E 
- $Z$ is the partition function $\left( Z = \sum_E \Omega(E) \cdot e^{- \beta E} \right)$
- $\Omega(E)$ is the multiplicity, *i.e.* the number of states with energy $E$
- $e^{- \beta E}$ is the Boltzmann factor $\left( \beta = \frac{1}{k_B T} \right)$

## Content of the notebook:
1. [Section 1](#1) **States Multiplicity**: investigate the number of states with the same velocity, *i.e.* the multiplicity $\Omega(v)$; 
2. [Section 2](#2) **Maxwell–Boltzmann Distribution**: how does the Maxwell–Boltzmann Distribution vary with temperature and particles-mass?;
3. [Section 3](#3) **Time Evolution of an Ideal Gas**: study of the velocity distribution of an ideal gas as function of time. How the Maxwell–Boltzmann Distribution emerges from a deterministic treatment, *i.e.* classical collision between particle?



In [1]:
# Imports 

import sys
import numpy as np
import math
import matplotlib as mpl
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
from IPython import display

In [2]:
matplotlib inline

In [3]:
# set up figures dimensions
plt.rcParams['figure.figsize'] = (6,4)
plt.rcParams['figure.dpi'] = 150

## 1) <a id='1'> </a> States Multiplicity

In order to understand the behaviour of $P(v)$, we need to take into account the multiplitcity $\Omega \left(\frac{1}{2} m v^2 \right) = \Omega(v)$, *i.e.* the number of ways a particle can move with velocity $v$. 

To determine $\Omega(v)$, we may presume the velocity states to be discrete and ask us: "how many of these states have velocity between $v$ and $v + \Delta v$?". 

The discrate states form a 3-D lattice of points $\left(v_x, v_y, v_z \right)$, with $v = \left| \vec{v}\right| = \sqrt{v_x^2 + v_y^2 + v_z^2}$. The states with velocity between $v$ and $v + \Delta v$ are these states in the spherical shell with inner radius $v$ and thickness $v + \Delta v$.

In [4]:
vmax = 10                 # Max velocity
Ncells = 5                # Number of lattice cells (odd number)
dv = vmax/((Ncells-1)/2)  # Delta velocity
Npoints1 = (Ncells)**3     # Number of lattice points
delta = 0.5               # Change in the interaction 
rbox = np.zeros((Npoints1,3))

# generate lattice points
count = 0
for i in range(- int((Ncells-1)/2), int((Ncells-1)/2)+1): # iterate over v_x, v_y, v_z
    for j in range(- int((Ncells-1)/2), int((Ncells-1)/2)+1):
        for k in range(- int((Ncells-1)/2), int((Ncells-1)/2)+1):
            rbox[count] = np.array([i,j,k])*dv # save the position of the lattice point
            count += 1 # move to the next point

@interact(v=(0,vmax - dv ,delta)) # interacting velocity 
def multiplicity(v):
    fig1 = plt.figure(figsize = (10,5)) # initialise the figure and set the dimension of the plot
    ax1 = fig1.add_subplot(121, projection = '3d') # add a 3D subplot

    # add labels and title
    ax1.set_xlabel('$v_x$') 
    ax1.set_ylabel('$v_y$')
    ax1.set_zlabel('$v_z$')
    
    # add axes
    ax1.plot([rbox[:,0].min() - 2, rbox[:,0].max() + 2], [0,0], [0,0], color = 'black')
    ax1.plot([0, 0], [rbox[:,0].min() - 2, rbox[:,0].max() + 2], [0, 0], color = 'black')
    ax1.plot([0, 0], [0,0], [rbox[:,0].min() - 2, rbox[:,0].max() + 2], color = 'black')
    
    # set limit of the axes
    ax1.set_xlim(-12, 12)
    ax1.set_ylim(-12, 12)
    ax1.set_zlim(-12, 12)
    
    # generate angles for sphere
    phi = np.linspace(0, 2*np.pi, 20)
    theta = np.linspace(0, np.pi, 20)
    
    # find points of the unit sphere
    x = np.outer(np.cos(phi), np.sin(theta))
    y = np.outer(np.sin(phi), np.sin(theta))
    z = np.outer(np.ones(np.size(phi)), np.cos(theta))
    
    # plot the spheres
    ax1.plot_surface(v*x, v*y, v*z, color='r', alpha = 0.1)                # inner sphere
    ax1.plot_surface((v+dv)*x, (v+dv)*y, (v+dv)*z, color='r', alpha = 0.1) # outer sphere
    
    #vel = np.sqrt(rbox[:,0]**2 + rbox[:,1]**2 + rbox[:,2]**2) # calulate v for the points
    vel1 = np.zeros(Npoints1)
    for i in range(Npoints1):
        vel1[i] = np.linalg.norm(rbox[i])
  
    # plot the lattice points
    for i in range(Npoints1):
        if v - 0.05 < vel1[i] and vel1[i] < v + dv - 0.05: # if they are inside the shell (-0.05 to include zero)
            ax1.scatter(rbox[i,0],rbox[i,1],rbox[i,2], s = 20, color ='r') # plot points 
        else:
            ax1.scatter(rbox[i,0],rbox[i,1],rbox[i,2], s = 10, color ='b') # plot points in different color
    # second subplot
    ax2 = fig1.add_subplot(122)
    
    # calulate the points of the plot
    X = np.arange(0,vmax - dv + delta , delta)
    Y = np.zeros(len(X))
    for i in range(len(X)):
        Y[i] = np.sum(np.where((vel1 > X[i] - 0.05) & (vel1 < X[i]+dv - 0.05) , 1, 0))
    
    # plot the points
    for i in range(len(X)):
        if X[i] == v:
            plt.scatter(X[i],Y[i], color = 'r')
        else:
            plt.scatter(X[i],Y[i], color = 'b')
    
    # set labels
    plt.xlabel('v')
    plt.ylabel('Number of states')
    plt.title('Plot of the multiplicity as function of the velocity')
    plt.grid();
    

interactive(children=(FloatSlider(value=2.5, description='v', max=5.0, step=0.5), Output()), _dom_classes=('wi…

The number of states scales with the volume of the shell: $v = 4 \pi v^2 \Delta v $

Let us rewrite P(v) as a distribution:

$$
P(v) = \frac{1}{Z} \cdot \Omega(v) \cdot e^{- \beta \frac{1}{2} m v^2 } \;\; \to \;\; P(v) \Delta v = \frac{1}{Z} \cdot \rho(v) \cdot e^{- \beta \frac{1}{2} m v^2 } \Delta v 
$$
where $P(v) \Delta v $ is the probability that particle has velocity between $v$ and $v + \Delta v$. 

$\rho(v) \Delta v $ is the number of states with velocity between $v$ and $v + \Delta v$, *i.e.* $\rho(v) \propto 4 \pi v^2$.

Considering arbitrary small velocity interval $dv$, we may rewrite this as:
$$
P(v) d v = \frac{4 \pi}{Z} v^2 e^{- \beta \frac{1}{2} m v^2 } d v
$$
where $Z = 4 \pi \int_0^{\infty} v^2 e^{- \beta \frac{1}{2} m v^2 } d v $ ensure that the probability distribution is normalized. This is know as **Maxwell–Boltzmann Distribution**.

## 2) <a id='2'> </a> Maxwell–Boltzmann Distribution

Making the substitution $x^2 = \beta \frac{1}{2} m v^2$:
$$
Z = 4 \pi \int_0^{\infty} v^2 e^{- \beta \frac{1}{2} m v^2 } d v = 4 \pi \left( \frac{2 k_B T}{m} \right)^{\frac{3}{2}} \int_0^{\infty} x^2 e^{- x^2 } d x 
$$
and knowing that $\int_{- \infty}^{\infty} x^2 e^{- x^2 } d x = \frac{\sqrt{\pi}}{4}$, it is possible to write $Z =  \left( \frac{2 \pi k_B T}{m} \right)^{\frac{3}{2}} $. This give the Maxwell–Boltzmann Distribution:

$$
P(v) d v = 4 \pi \left( \frac{m}{2 \pi k_B T}\right)^{\frac{3}{2}} v^2 e^{-\frac{ m v^2 }{2 k_B T}} d v
$$

In [5]:
def Maxwell_Boltzmann(T,m,v):
    """
    Calculate the Maxwell–Boltzmann Distribution.
    Inputs:
    T      Temperature
    m      Mass of the particle
    v      Velocities (array-like)
    Output: Probability distribution
    """
    return 4*np.pi*(m/(2*np.pi*k*T))**(3/2)*v**2*np.e**(-(m*v**2)/(2*k*T))

In [6]:
m_u = 1.66e-27 # Dalton (Kg)
k = 1.38064852e-23 # m2 kg s-2 K-1 (Boltzmann constant)

v_min = 0
v_max = 1500
Npoints2 = 200
vel2 = np.linspace(v_min, v_max, Npoints2)

@interact(T=(60,400,20), m=(5,50,5)) # interacting velocity 
def plot_distribution(T,m):
    fig1 = plt.figure(figsize = (10,5)) # initialise the figure and set the dimension of the plot
    
    plt.plot(vel2, Maxwell_Boltzmann(T,m*m_u,vel2), label='Distribution') # plot the distribution
    
    # calulate and plot the mean and most-probable velocities 
    vmean = np.sqrt(3*k*T/(m*m_u))
    vprob = np.sqrt(2*k*T/(m*m_u))
    plt.axvline(x=vmean, ymin = 0, ymax =  Maxwell_Boltzmann(T,m*m_u,vmean)*200,
                color='g', label='mean velocity')
    plt.axvline(x=vprob, ymin = 0, ymax =  Maxwell_Boltzmann(T,m*m_u,vprob)*200,
                color='r', label='most-probable velocity')
    
    # add features to the plot
    plt.xlim(v_min,v_max)
    plt.ylim(0, 0.005)
    plt.xlabel('Velocity (m/s)')
    plt.ylabel('Probability Density (s/m)')
    plt.title('Maxwell–Boltzmann Distribution')
    plt.grid()
    plt.legend();
    

interactive(children=(IntSlider(value=220, description='T', max=400, min=60, step=20), IntSlider(value=25, des…

<font color=red>Note: I am not sure about the normalization of the distribution. The y-axis seems to have too low values. However, I found graphs with this order of magintude as well. 
</font> 

## 3) <a id='3'> </a> Time Evoution of an Ideal Gas 
The Maxwell–Boltzmann Distribution appears as an emerging behavior of *"mean"* probability distribution. The velocity distribution tends to a Maxwell–Boltzmann Distribution a) as the number of particles and b) as the running-time increses. Let us see this with two different simulations.

### a) Velocity Distribution as function of number of particles

In the following subsection, the instantaneous velocity probability is plotted at a time t, *i.e.* the probability of finding a particle with a velocity between $v$ and $v + \Delta v$ at the time t. 

Thus, let us create a gas of N ideal particle with the same kinetic energy and with random velocity direction, and investigate how the system evolves. The particles collision with the walls and between themselves are modeled as perfectly elastic collisions. The collisions between particles change their velocities as time passes. As the number of particles increases, an emerging behavior appears: the Maxwell–Boltzmann Distribution. With a higher number of particles the random fluctuations become less relevant and the distribution approaches the Maxwell–Boltzmann Distribution.

The initial velocity of the particle is given by:

$$
\frac{1}{2}m \overline{v}^2 = \frac{3}{2} k_B T  
$$

<font color=red>Note: Working in progress.
</font> 

In [7]:
def generate(N, R, Nsteps, T, boxlen):
    """
    Generate N particles at T in a random position and with a random velocity direction.
    Inputs:
    N        Number of particles
    R        Radius of the particles
    Nsteps   Number of steps of the simulation
    T        Temperature 
    boxlen   Lenght of the box
    Outputs: position and velocity of the partcles
    """
    v0 = np.sqrt(T)/2 # velocity proportional to the root of the temperature 
    # generate array for storing position and velocities over the simulation
    r = np.zeros((Nsteps, N, 2)) 
    v = np.zeros((Nsteps, N, 2))

    r[0,0] = (boxlen - 2*R) *np.random.rand(2) + R # random initial positions of the first particle
    
    for i in range(1,N): # for every other particle
        over = True
        while over == True:
            r0 = (boxlen - 2*R) *np.random.rand(2) + R
            dr = r[0] - r0
            dis = np.sqrt(dr[:,0]**2 + dr[:,1]**2) 
            if np.min(dis) > 2*R: # check if the particl overlap with the others
                 over = False
        r[0,i] =  r0 # random initial positions
    
    # reandom velocity directions
    thetas = 2*np.pi*np.random.rand(N) 
    v[0,:,0] = np.cos(thetas)*v0
    v[0,:,1] = np.sin(thetas)*v0
    
    return r, v

In [8]:
def hardwall(r,v,boxlen):
    """Implements hard-wall boundary conditions for a particle
    Inputs:
    r       Positions of the particles
    v       Velocities of the particles
    boxlen  Lenght of the box
    Outputs: position and veloicity of the paritcles after the collision
    """
    rout = r
    vout = v
    for j in range(len(r)): # for each particle 
        if r[j,0] <= R:
            if v[j,0] < 0: 
                vout[j,0] = - v[j,0]
        if r[j,0] >= boxlen - R:
            if v[j,0] > 0: 
                vout[j,0] = - v[j,0]

        if r[j,1] <= R:
            if v[j,1] < 0: 
                vout[j,1] = - v[j,1]
        if r[j,1] >= boxlen - R:
            if v[j,1] > 0: 
                vout[j,1] = - v[j,1]
            
    return rout, vout

In [9]:
def overlap(r, v, N, R, dt):
    """
    Check if one particle overlap and update the velocities
    """
    for i in range(N):
        for j in range(i+1, N): #check for overlap 
            dr1 =  r[i] - r[j]
            dis1 = np.sqrt(dr1[0]**2 + dr1[1]**2) 
            if dis1 < 2*R:
                dr2 = - r[j] - v[j]*dt + r[i] + v[i]*dt
                dis2 = np.sqrt(dr2[0]**2 + dr2[1]**2) 
                if dis1 > dis2:
                    norm = dr1/dis1
                    v1 = v[i] - np.inner(v[i] - v[j], norm)*norm
                    v2 = v[j] - np.inner(v[j] - v[i], norm)*norm

                    v[i] = v1
                    v[j] = v2
                    #v[j] = collision(dr1, v[i], v[j]) # if they collide, change the velocity
                #r[i] = r[j] - 2.2*R*dr/dis
    return v 

In [10]:
def velocity_density(v, Nsteps, N, Ndiag):
    """
    Calculate the velocity probabiliyt density for the disposition
    """
    vel = np.zeros((Nsteps,N))
    for i in range(Nsteps):
        for j in range(N):
            vel[i,j] = np.sqrt(v[i,j,0]**2 + v[i,j,1]**2)

    vel_den = np.zeros((Nsteps,Ndiag))
    vel_max =  np.max(vel)
    vel_step = vel_max/Ndiag
    
    for i in range(Nsteps):
        for j in range(N):
            for k in range(Ndiag):
                if (vel_step * k) <= vel[i,j]:
                    if vel[i,j] <= (vel_step * (k + 1)):
                        vel_den[i,k] = vel_den[i,k] + 1 
    
    vel_den = vel_den/N
    return vel_den, vel_max

In [11]:
N = 15
boxlen = 1
R = boxlen/30
T = 300
Nsteps = 500
Ndiag = 8
dt = 0.001
#

r, v = generate(N,R, Nsteps, T, boxlen)
## Velocity Verlet algorithm
for i in range(1,Nsteps):
    v[i-1] = overlap(r[i-1], v[i-1], N, R, dt) # check collisions
    r[i] = r[i-1] + dt*v[i-1] # update position
    r[i], v[i] = hardwall(r[i],v[i-1],boxlen) # boundary conditions
    v[i] = v[i-1] #update velocitie

vel_den, vel_max = velocity_density(v, Nsteps, N, Ndiag)
